In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import numpy as np

import bvcopula
import utils
import select_copula

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
animal = 'ST263'
dayN = 4
day_name = 'Day{}'.format(dayN)
path = '/home/nina/VRData/Processing/pkls'

n1 = 1
n2 = 12

X,Y = utils.load_experimental_data(path, animal, day_name, n1, n2)
exp_pref = '{}_{}'.format(animal,day_name)

In [ ]:
select_copula.select_copula_model(X,Y,0,exp_pref,'./imgs',n1,n2)

Trying  Gaussian 
Converged in 400 steps!
Gaussian  took  15  sec
WAIC=2.67e+03
Trying  Frank 
Converged in 300 steps!
Frank  took  9  sec
WAIC=2.73e+03
Trying  Clayton 0°


In [ ]:
Y.shape